In [13]:
import os
import sys
import copy
import json
import argparse
import subprocess

import mysql.connector

from itertools import chain, combinations

In [14]:
sys.path.insert(0, "../")

In [8]:
from definitions import *
import api

In [15]:
CHESSDB_HOST = "localhost"
CHESSDB_USER = "chess_master"
CHESSDB_PASSWORD = "qwerty"
CHESSDB_PORT = "3306"
CHESSDB_NAME = "CHESS_DB"

chessApi = None
try:
    chessApi = api.CHESS_DB_API(CHESSDB_HOST, CHESSDB_USER, CHESSDB_PASSWORD, CHESSDB_NAME, CHESSDB_PORT)
    chessApi.connect()
except:
    print("Failed to connect to database")
    exit(1)

Connected to MySQL database


In [10]:
# Example query
query = '''SELECT * FROM Transcripts'''
result = chessApi.execute_query(query)
result
# chessApi.disconnect() # Preform when done working with the notebook


hi!
Disconnected from MySQL database


[(1,
  'GRCh38',
  'chr18',
  0,
  47221,
  49615,
  '47221-48447,48940-49050,49129-49237,49501-49615',
  datetime.datetime(2023, 10, 12, 2, 45, 35)),
 (2,
  'GRCh38',
  'chr18',
  1,
  158557,
  214629,
  '158557-158714,163308-163453,166787-166819,178933-179037,180236-180339,192842-192900,196637-196767,197616-197696,198047-198132,199202-199316,202880-202945,203098-203190,204564-204692,209971-210031,210386-210493,211133-214629',
  datetime.datetime(2023, 10, 12, 2, 45, 35)),
 (3,
  'GRCh38',
  'chr18',
  0,
  214520,
  268047,
  '214520-214921,215429-215504,216486-216633,218886-218969,223440-223505,224084-224179,224924-224994,225089-225139,225337-225403,226801-226901,246324-246455,247849-247957,252539-252612,254273-254355,259180-259275,259682-259730,260186-260304,264026-264092,265303-265363,265457-265530,267966-268047',
  datetime.datetime(2023, 10, 12, 2, 45, 35)),
 (4,
  'GRCh38',
  'chr18',
  0,
  316737,
  500701,
  '316737-320064,321662-321807,331668-331777,333007-333143,334742-33

In [ ]:
# to_gtf_with_attributes function
# Goal: output a .GTF file for a given source name with its attributes, expanding on to_gtf function
from collections import defaultdict

# FIXED FORMATTING ATTRIBUTES AND ADDING EXONS TO THE OUTPUT!

#-----------------------------------------------------
# INPUT: SELECT SOURCE NAME
source_name = "RefSeq"
#-----------------------------------------------------

# Get the attributes for each transcript with the inputted source name
# Description: queries the Sources table by a given name, joins all the sources with that name to the TxDBXREF table by their source ID, and then uses the tid, sourceID, and transcript_id to query the attributes table, outputting the result; also gets info from Transcripts table now
# Output table:
# indexes 0     1       2     3         4         5     6     7     8     9      10
# values tid sourceId tx_id atr_name attr_value seqId  start  end  strand score  exons
query =  f"SELECT a.*, t.sequenceID, t.start, t.end, t.strand, tx.score, t.exons FROM Attributes a JOIN TxDBXREF tx ON a.tid = tx.tid AND a.sourceID = tx.sourceID AND a.transcript_id = tx.transcript_id JOIN Transcripts t ON tx.tid = t.tid JOIN Sources s ON a.sourceID = s.sourceID WHERE s.name = '{source_name}'"

# Execute Query
select_res = chessApi.execute_query(query)
select_res

transcript_data = defaultdict(list) # automatically initializes a list for each key
for row in select_res:
    transcript_id = row[0]  # Assuming transcript_id is in the first position
    transcript_data[transcript_id].append(row)

# Writing the GTF file
# Output File Name
outfname = "new_gtf_new"
with open(outfname, "w") as outFP:
    if select_res is None or not select_res:
        print(f"No transcripts found for assembly {source_name}.")
        # write a dummy transcript to avoid errors downstream
        outFP.write("		transcript	0	0	.	+	.	transcript_id \"nan\";\n")
        outFP.write("		exon	0	0	.	+	.	transcript_id \"nan\";\n")
    else:
        for transcript_id, rows in transcript_data.items():
            gtf_str = ""
            # Construct transcript line
            attributes = "; ".join(f"{row[3]} {row[4]}" for row in rows)
            # print(attributes + "\n")
            strand = "+" if rows[0][8] == 1 else "-"
            gtf_str = f"{rows[0][5]}\t{source_name}\ttranscript\t{rows[0][6]}\t{rows[0][7]}\t.\t{strand}\t.\ttranscript_id \"{transcript_id}\"; {attributes}\n"

            outFP.write(gtf_str)

            # construct exon lines
            gtf_str = ""
            for exon in rows[0][10].split(","):
                exon_start, exon_end = [str(int(v)) for v in exon.split("-")]
                gtf_str += rows[0][5]+"\t"+str(source_name)+"\texon\t"+exon_start+"\t"+exon_end+"\t.\t"+strand+"\t.\ttranscript_id \""+str(rows[0][0])+"\";\n"
            outFP.write(gtf_str)


In [17]:
# OLD CODE

# to_gtf_with_attributes function
# Goal: output a .GTF file for a given source name with its attributes;
#       resulting gtf can have a number of source_name(s) and resolves ties via a priority list
from collections import defaultdict


ranks = ["CHESS.3.0", "RefSeq", "MANE"]


#------------THE CHANGES I HAVE TO MAKE---------------
# do all in one SQL query
# conditional join, you can try for loop in SQL
# Simpler solution: Query for everything in the attribute table, iterate through it and resolve conflicts as the appears; do this in all one SQL line or using some python
#-----------------------------------------------------


# This way is too slow
# Initialize an empty dictionary to store query results for each source
result_tables = {}

for source_name in ranks:
    query =  f"SELECT a.*, t.sequenceID, t.start, t.end, t.strand, tx.score FROM Attributes a JOIN TxDBXREF tx ON a.tid = tx.tid AND a.sourceID = tx.sourceID AND a.transcript_id = tx.transcript_id JOIN Transcripts t ON tx.tid = t.tid JOIN Sources s ON a.sourceID = s.sourceID WHERE s.name = '{source_name}'"
    output = chessApi.execute_query(query)
    result_tables[source_name] = output

# Merge the tables and resolve conflicts based on priority
final_table = {}
for source_name in ranks:
    for row in result_tables[source_name]: # each row is a tuple in result_tables[source_name]
        tid = row[0]
        if tid not in final_table or final_table[tid]['priority'] < ranks.index(source_name):
            final_table[tid] = {'data': row, 'priority': ranks.index(source_name)}

processed_data = [value['data'] for value in final_table.values()]

# indexes 0     1       2     3         4         5     6     7     8     9      10
# values tid sourceId tx_id atr_name attr_value seqId  start  end  strand score  exons


# Write GTF File
# Output File Name
outfname = "example_gtf_with_priorities"
# Writing the File
with open(outfname,"w") as outFP:
    if processed_data is None or not processed_data:
        print(f"No transcripts found for assembly {sources_name}.")
        # write a dummy transcript to avoid errors downstream
        outFP.write("		transcript	0	0	.	+	.	transcript_id \"nan\";\n")
        outFP.write("		exon	0	0	.	+	.	transcript_id \"nan\";\n")
    else:
        for row in processed_data:
            gtf_str = ""
            # construct transcript line
            strand = "+" if row[8] == 1 else "-"
            gtf_str += row[5]+"\t"+str(source_name)+"\ttranscript\t"+str(row[6])+"\t"+str(row[7])+"\t.\t"+strand+"\t.\ttranscript_id \""+str(row[0])+"\""+"\t"+str(row[3])+"\t"+str(row[4])+";\n"
            outFP.write(gtf_str)



hi!
hi!
hi!


In [23]:
# NEW CODE:

# Pseudocode for the SQL query
# tid     source            akey   avalue
# 1       "CHESS"        "key1"      "val4"
# 2       "CHESS"        "key1"      "val4"
# 1       "MANE"        "key1"      "val2"
# 1       "CHESS"        "key1"      "val4"
# 1       "MANE"        "key1"      "val2"
# 2       "CHESS"        "key1"      "val4"
# 1       "MANE"        "key1"      "val2"
# 1       "CHESS"        "key1"      "val4"
# 2       "CHESS"        "key1"      "val4"
# select first(*) from table order by tid, akey, ORDER BY FIELD(source, 'MANE', 'CHESS', 'RefSeq');
#---------------------------------------------------------------------------------------------------

from collections import defaultdict


# Define the list of source names for filtering
source_names = ['CHESS.3.0', 'RefSeq', 'MANE']

# Create the SQL query string
# SQL query using a subquery to reference the newAttributes table
query = '''SELECT na.*, t.sequenceID, t.start, t.end, t.strand, tx.score, t.exons 
FROM (
    SELECT *
    FROM (
        SELECT a.*, s.name AS source_name,
            ROW_NUMBER() OVER(PARTITION BY a.tid, a.name ORDER BY a.tid, a.name,
                CASE
                    WHEN s.name = 'CHESS.3.0' THEN 0
                    WHEN s.name = 'RefSeq' THEN 1
                    WHEN s.name = 'MANE' THEN 2
                    ELSE 3  -- Place other sources at the end
                END) AS rn
        FROM Attributes a
        JOIN Sources s ON a.sourceID = s.sourceID
    ) AS ranked
    WHERE rn = 1 AND (source_name = 'CHESS.3.0' OR source_name = 'RefSeq' OR source_name = 'MANE')
) na 
JOIN TxDBXREF tx ON na.tid = tx.tid AND na.sourceID = tx.sourceID AND na.transcript_id = tx.transcript_id 
JOIN Transcripts t ON tx.tid = t.tid'''

# Execute Query
select_res = chessApi.execute_query(query)
select_res

transcript_data = defaultdict(list) # automatically initializes a list for each key
for row in select_res:
    transcript_id = row[0]  # Assuming transcript_id is in the first position
    transcript_data[transcript_id].append(row)

# Writing the GTF file
# Output File Name
outfname = "new_results_gtf"
with open(outfname, "w") as outFP:
    if select_res is None or not select_res:
        print(f"No transcripts found for assembles {source_names}.")
        # write a dummy transcript to avoid errors downstream
        outFP.write("		transcript	0	0	.	+	.	transcript_id \"nan\";\n")
        outFP.write("		exon	0	0	.	+	.	transcript_id \"nan\";\n")
    else:
        for transcript_id, rows in transcript_data.items():
            gtf_str = ""
            # Construct transcript line
            attributes = "; ".join(f"{row[3]} {row[4]}" for row in rows)
            # print(attributes + "\n")
            strand = "+" if rows[0][10] == 1 else "-"
            gtf_str = f"{rows[0][7]}\t{rows[0][5]}\ttranscript\t{rows[0][8]}\t{rows[0][9]}\t.\t{strand}\t.\ttranscript_id \"{rows[0][2]}\"; {attributes}\n"

            outFP.write(gtf_str)

            # construct exon lines
            gtf_str = ""
            for exon in rows[0][12].split(","):
                exon_start, exon_end = [str(int(v)) for v in exon.split("-")]
                gtf_str += rows[0][7]+"\t"+str(rows[0][5])+"\texon\t"+exon_start+"\t"+exon_end+"\t.\t"+strand+"\t.\ttranscript_id \""+str(rows[0][2])+"\";\n"
            outFP.write(gtf_str)


            # indexes 0     1       2     3         4           5          6       7     8     9      10    11     12
            # values tid sourceId tx_id atr_name attr_value source_name sourceID seqId  start  end  strand score  exons



hi!


In [21]:
query = '''SELECT na.*, t.sequenceID, t.start, t.end, t.strand, tx.score, t.exons 
FROM (
    SELECT *
    FROM (
        SELECT a.*, s.name AS source_name,
            ROW_NUMBER() OVER(PARTITION BY a.tid, a.name ORDER BY a.tid, a.name,
                CASE
                    WHEN s.name = 'CHESS.3.0' THEN 0
                    WHEN s.name = 'RefSeq' THEN 1
                    WHEN s.name = 'MANE' THEN 2
                    ELSE 3  -- Place other sources at the end
                END) AS rn
        FROM Attributes a
        JOIN Sources s ON a.sourceID = s.sourceID
    ) AS ranked
    WHERE rn = 1 AND (source_name = 'CHESS.3.0' OR source_name = 'RefSeq' OR source_name = 'MANE')
) na 
JOIN TxDBXREF tx ON na.tid = tx.tid AND na.sourceID = tx.sourceID AND na.transcript_id = tx.transcript_id 
JOIN Transcripts t ON tx.tid = t.tid'''

# Execute Query
select_res = chessApi.execute_query(query1)
select_res


hi!


[(1,
  4,
  'CHS.128994.1',
  'assembly_id',
  'ALL_11735693',
  'CHESS.3.0.1',
  1,
  'chr18',
  47221,
  49615,
  0,
  0,
  '47221-48447,48940-49050,49129-49237,49501-49615'),
 (1,
  1,
  'ENST00000308911.9',
  'CDS_db_xref',
  'RefSeq:NP_001345618.1',
  'GENCODE',
  1,
  'chr18',
  47221,
  49615,
  0,
  0,
  '47221-48447,48940-49050,49129-49237,49501-49615'),
 (1,
  2,
  'rna-NM_001358689.2',
  'CDS_Dbxref',
  'CCDS:CCDS86657.1,Ensembl:ENSP00000496713.1,GeneID:260334,Genbank:NP_001345618.1,HGNC:HGNC:24983',
  'GENCODE',
  1,
  'chr18',
  47221,
  49615,
  0,
  0,
  '47221-48447,48940-49050,49129-49237,49501-49615'),
 (1,
  2,
  'rna-NM_001358689.2',
  'CDS_gbkey',
  'CDS',
  'CHESS.3.0.1',
  1,
  'chr18',
  47221,
  49615,
  0,
  0,
  '47221-48447,48940-49050,49129-49237,49501-49615'),
 (1,
  2,
  'rna-NM_001358689.2',
  'CDS_Name',
  'NP_001345618.1',
  'CHESS.3.0.1',
  1,
  'chr18',
  47221,
  49615,
  0,
  0,
  '47221-48447,48940-49050,49129-49237,49501-49615'),
 (1,
  2,
  'rna

In [ ]:
query2 = '''SELECT na.*, t.sequenceID, t.start, t.end, t.strand, tx.score, t.exons 
FROM (
    SELECT *
    FROM (
        SELECT a.*, s.name AS source_name,
            ROW_NUMBER() OVER(PARTITION BY a.tid, a.name ORDER BY a.tid, a.name, FIELD(s.name, 'CHESS.3.0', 'RefSeq', 'MANE')) AS rn
        FROM Attributes a
        JOIN Sources s WHERE a.sourceID IN (
            SELECT sourceID
            FROM Sources
            WHERE name IN ('CHESS.3.0', 'RefSeq', 'MANE')
        )
    ) AS ranked
    WHERE rn = 1
) na 
JOIN TxDBXREF tx ON na.tid = tx.tid AND na.sourceID = tx.sourceID AND na.transcript_id = tx.transcript_id 
JOIN Transcripts t ON tx.tid = t.tid 
'''





'''SELECT a.*, t.sequenceID, t.start, t.end, t.strand, tx.score, t.exons
FROM (
    SELECT a.*
    FROM Attributes a
    WHERE a.sourceID IN (
        SELECT sourceID
        FROM Sources
        WHERE name IN ('CHESS.3.0', 'RefSeq', 'MANE')
    # )
    # AND NOT EXISTS (
    #     SELECT 1
    #     FROM Attributes a2
    #     WHERE a.tid = a2.tid
    #     AND a2.sourceID IN (
    #         SELECT sourceID
    #         FROM Sources
    #         WHERE name IN ('CHESS.3.0', 'RefSeq', 'MANE')
    #     )
    #     AND (
    #         (a2.sourceID = 'CHESS.3.0' AND a.sourceID IN ('RefSeq', 'MANE')) OR
    #         (a2.sourceID = 'RefSeq' AND a.sourceID = 'MANE')
    #     )
    # )
) a
JOIN TxDBXREF tx ON a.tid = tx.tid AND a.sourceID = tx.sourceID AND a.transcript_id = tx.transcript_id
JOIN Transcripts t ON tx.tid = t.tid
JOIN Sources s ON a.sourceID = s.sourceID
WHERE s.name IN ('CHESS.3.0', 'RefSeq', 'MANE')'''





# a table with the attributes for all the inputed sources as well as a source_rank column saying the rank of that attribute row
query2 = '''SELECT a.*
FROM Attributes a
WHERE a.sourceID IN (
    SELECT sourceID
    FROM Sources
    WHERE name IN ('CHESS.3.0', 'RefSeq', 'MANE')
)
AND NOT EXISTS (
    SELECT 1
    FROM Attributes a2
    WHERE a.tid = a2.tid
    AND a2.sourceID IN (
        SELECT sourceID
        FROM Sources
        WHERE name IN ('CHESS.3.0', 'RefSeq', 'MANE')
    )
    AND (
        (a2.sourceID = 'CHESS.3.0' AND a.sourceID IN ('RefSeq', 'MANE')) OR
        (a2.sourceID = 'RefSeq' AND a.sourceID = 'MANE')
    )
);
'''
